# functools.singledispatch for Network Engineers
 

This blog will cover how to solve polymorphism use cases using the singledispatch decorator.  

In the past I would sove this this problem using solution 1 but then slowly eveolved into the Solution 2. Solution 3 will be my new go to and Ill explain why below.

First lets import the packages that we will need for this demo. I will be using dataclasses in this example. If you are looking to recreate this code, Python 3.7 will be required.    

In [1]:
from dataclasses import dataclass
from functools import singledispatch

## Set Up

Next lets define the models we will use for the Demo. In this example I will be explaining how to handle diffrent router types.  The varaints of this base type Router will be CiscoRouter and JuniperRouter. This simulates what Network Engineers would need to prepare for in the our domain.


lets create our dataclasses(click, if you are not familair with, [Data classes](https://docs.python.org/3/library/dataclasses.html). 

The first class will be the base class Router. This router will be inhertited by the CiscoRouter and JuniperRouter classes. 

A CiscoRouter or JuniperRouter can be of type Router, but a JuiperRouter cannot be of type CiscoRouter or vice versa. 

Next we will define 2 handlers, one for cisco router and another for Juniper routers. 

In [27]:
@dataclass
class Router:
    hostname: str
    vendor: str = "unknown"

@dataclass
class CiscoRouter(Router):
    vendor: str = "cisco"


@dataclass
class JuniperRouter(Router):
    vendor: str = "juniper"
 

# Handlers
def _handle_cisco(router: CiscoRouter):
    print("Handling Cisco Router")

    
def _handle_juniper(router: JuniperRouter):
    print("Handling Juniper Router")
    
def _handle_router(router: Router):
    print("Handling Generic router")

In [28]:
routers = [
    CiscoRouter("R1"), 
    JuniperRouter("R2"),
    Router("R3")
]

##  Solution 1

We will define 1 function and 2 private functions. In python visibity is assumed by name of the function and the value '_' underscore prepended on a function indicates that function is private. 

the _handle_cisco and _handle_juinper functions, the will expect an instance of the respective rotuer type. When using this API, the user will only call the handle_router function that is used determine which handler will need to be called. 

In this example, is check the instance of the router, and if it of the specefiied instance, the correct handler is called

### Benefits

### Drawbacks
- Everytime a new device is supportted, a new conditional and handler needs to be provisioned, if you support hundreds of routers, this task can be very hard to maintain and scale

In [35]:
def handle_router(router: Router): # generic function
    if isinstance(router, Router): 
        return _handle_router(router)
    
    if isinstance(router, CiscoRouter): # never executes matters
        return _handle_cisco(router)
        
    if isinstance(router, JuniperRouter): # never executes matters
        return _handle_juniper(router)
    
    
    

for router in routers:
    handle_router(router)

Handling Generic router
Handling Generic router
Handling Generic router


In [36]:
def handle_router(router: Router): # generic function
    if isinstance(router, CiscoRouter):
        return _handle_cisco(router)
        
    if isinstance(router, JuniperRouter):
        return _handle_juniper(router)
    
    if isinstance(router, Router): # position matters, default catch all
        return _handle_router(router)
    

for router in routers:
    handle_router(router)

Handling Cisco Router
Handling Juniper Router
Handling Generic router


## Solution 2

In this soution will use dictionary to preform looks up for the correct handler.  We will define an handler_map that will use the vendors name as the key and the handler function as the value. 

This solution is simple, once the handle_router_v2 function is called, a dictionary look up is preformed, if a match is not found, the _handle_cisco handler will be used. 

### Benefits

### Drawbacks

In [37]:
handler_map = {
    "cisco": _handle_cisco,
    "juniper": _handle_juniper
}

def handle_router_v2(router: Router): # generic function
    _handler = handler_map.get(router.vendor, _handle_router)
    return _handler(router)

    
for router in routers:
    handle_router_v2(router)


Handling Cisco Router
Handling Juniper Router
Handling Generic router


## Solution 3

IN Solution 3, We will explore the use of the singledispatch utility decorator in the functools package. in this example we will define what process_router function. This function is used by the consumers of our API in order to make the call the process the apporiiated router. 

The 2 functions below the process router definition may not see like working functions but this is where the fool factor of the singledispatch occurs. 

1. @singledispatch decorator is used on the main function to indicate the activation of the singledispatch feature.
2. the functions name "_" indicates a nameless function. The way the function calls are handled is by the @func_name.register decorator. the way single dispatch knows what type needs to map to a single nameless function is by the use of TypeHint. 

### Benefits

### Drawbacks

In [40]:
@singledispatch
def handle_router_v3(router: Router): # generic function
    print("Handling  Generic Router")


@handle_router_v3.register
def _(router: CiscoRouter):
    print("Handling  Cisco Router")


@handle_router_v3.register
def _(router: JuniperRouter):
    print("Handling  Juniper Router")

for router in routers:
    handle_router_v3(router)


Handling  Cisco Router
Handling  Juniper Router
Handling  Generic Router
